# Supervised Machine Learning - Test Your Knowledge Key

In [1]:
library(readxl);
library(lubridate);
library(tidyverse);
library(gtsummary);
library(flextable);
library(caret);
library(randomForest);


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.3     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.3     ✔ tibble  3.2.1
✔ purrr   1.0.2     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘flextable’


The following objects are masked from ‘package:gtsummary’:

    as_flextable, continuous_summary


The following object is masked from ‘package:purrr’:

    compose


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


randomForest 4.7-1.1

Type rfNews() to see new features/c

In [2]:
# Load the data
manganese_data <- data.frame(read_excel("Module5/Module5_Manganese_Data.xlsx"))

# View the top of the dataset
head(manganese_data) 

,Tax_ID,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Longtitude,Latitude,Stream_Distance,Elevation,Detect_Concentration
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1006004,9/24/12,52,165,41,60.0,7.7,-80.29918,35.17970,811.9613,611.499,D
2,1024009,12/17/15,40,445,42,2.0,7.3,-80.31061,35.15487,341.7654,688.452,D
3,1054019,2/2/15,45,160,40,40.0,7.4,-80.33170,35.16158,634.2669,692.789,D
4,1057017,10/22/12,42,440,57,1.5,8.0,-80.32263,35.13962,855.4713,692.789,ND
5,1060006,1/3/11,48,120,42,25.0,7.1,-80.32911,35.13027,163.5688,683.387,D
6,1066006,12/15/15,60,280,32,10.0,8.2,-80.32205,35.10479,725.7672,614.599,D


Like we did in the module, we'll start by changing some of the data types. 

In [3]:
manganese_data = manganese_data %>%
    # Converting `Detect_Concentration from a character to a factor
    mutate(Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
        # converting water sample date from a character to a date type 
        Water_Sample_Date = mdy(Water_Sample_Date %>%
    # Removing tax id and only keeping the predictor and outcome variables in the dataset
    select(-Tax_ID)

head(manganese_data)

ERROR: Error in parse(text = x, srcfile = src): <text>:9:1: unexpected symbol
8: 
9: head
   ^


Testing for differences in predictor variables acrosss the outcome classes.

In [ ]:
manganese_data %>%
    tbl_summary(by = Detect_Concentration,
    # Selecting columns to include
    #include = colnames(manganese_data[c(2:11)]), 
    # Displaying the mean and standard deviation in parantheses for all continuous variables
                statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
    # Adding a column that displays the total number of samples for each variable
    # This will be 713 for all variables since we have no missing data
    add_n() %>% 
    # Adding a column that displays the p value from anova
    add_p(test = list(all_continuous() ~ "aov")) %>% 
    #as_flex_table() %>%
    #bold(bold = TRUE, part = "header")
    as_tibble()

4 predictor variables (casing depth, pH, longtitude, and latitude) are significantly different, so the model should be able to predict moderately well. 

Next, setting up cross validation and parameters to be tuned.

In [ ]:
set.seed(12)

manganese_index = createFolds(manganese_data$Detect_Concentration, k = 5) 

ntree_values = c(50, 250, 500) # number of trees 
p = dim(manganese_data)[2] - 1 # number of predictor variables in the dataset
mtry_values = c(sqrt(p), p/2, p) # number of predictors to be used in the model

Predicting with RF

In [ ]:
# Setting the seed again so the predictions are consistent
set.seed(12)

# Creating an empty dataframe to save the metrics
metrics_df = data.frame()

# Iterating through the cross validation folds
for (i in 1:length(manganese_index)){
    # Training data
    data_train = manganese_data[-manganese_index[[i]],]
    
    # Test data
    data_test = manganese_data[manganese_index[[i]],]
    
    # Creating empty lists and dataframes to store errors 
    reg_rf_pred_tune = list()
    rf_OOB_errors = list()
    rf_error_df = data.frame()
    
    # Tuning parameters: using ntree and mtry values to determine which combination yields the smallest OOB error 
    # from the validation datasets
    for (j in 1:length(ntree_values)){
        for (k in 1:length(mtry_values)){
            
            # Running RF to tune parameters
            reg_rf_pred_tune[[k]] = randomForest(Detect_Concentration ~ ., data = data_train, 
                                                 ntree = ntree_values[j], mtry = mtry_values[k])
            # Obtaining the OOB error
            rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                   "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
            
            # Storing the values in a dataframe
            rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
        }
    }
    
    # Finding the lowest OOB error using best number of predictors at split
    best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

    # Now running RF on the entire training set with the tuned parameters
    reg_rf <- randomForest(Detect_Concentration ~ ., data = data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

    # Predicting on test set and adding the predicted values as an additional column to the test data
    data_test$Pred_Detect_Concentration = predict(reg_rf, newdata = data_test, type = "response")

    # Obtaining the confusion matrix
    matrix = confusionMatrix(data = data_test$Pred_Detect_Concentration, 
                             reference = data_test$Detect_Concentration, positive = "1")
    
    # Extracting balanced accuracy, sensitivity, specificity, and PPV
    matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])))
    
    # Adding values to df to be averaged across the 5 splits from CV
    metrics_df = rbind(metrics_df, matrix_values)
}

# Taking average
metrics_df = metrics_df %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
          Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value))

# Viewing the model's performance metrics
metrics_df

Takeaways from this confusion matrix:

+ Overall, the model performed moderately well at predicting if iAs would be detected based on a balanced accuracy of ~0.7
+ RF did an okay job of predicting detect data with a sensitivity of 0.6 and a PPV of ~0.7
+ The model was better at predicting non-detect data based on a specificity of ~0.8 